In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

/home/maroco/anaconda3/envs/AIML2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
##GPU 사용 시
device = torch.device("cuda:0")

In [3]:
#bert 모델, vocab 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /home/maroco/SMS-Filtering/lib/.cache/kobert_v1.zip
using cached model. /home/maroco/SMS-Filtering/lib/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [4]:
dataset_train = nlp.data.TSVDataset('/home/maroco/dataset/model/train_set.tsv', field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset('/home/maroco/dataset/model/test_set.tsv', field_indices=[1,2], num_discard_samples=1)

In [5]:
dataset_train[0]

['쿠캣 인기상품 원 타임딜이 지금 시작됐어요쿠캣의 찐 가족이라면 쿠캣데이를 절대 놓치지 마세요오직 최명희 고객님을 위해 준비한 월 쿠캣데이가 지금 시작됐어요쿠캣데이 기획전 전 상품  추가 할인 쿠폰도 확인하고 오늘 딱 하루 쿠캣데이 혜택을 절대 놓치지 마세요 쿠캣데이 혜택 인기상품 원 타임딜 최대  상품 할인  추가 할인 쿠폰 선착순 아이스박스 무료 증정쿠캣데이 BEST 쿠캣 볶음밥 종 쿠캣 밥도둑 종 쿠캣 납작막창 종 쿠캣 렌지용 화덕 생선구이 종 띵커바디 식단관리 도시락httpspagecookatmarketcomsqhl무료거부 ',
 '1']

In [6]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [7]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [8]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
#BERTDataset 클래스 이용, TensorDataset으로 만들어주기
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /home/maroco/SMS-Filtering/lib/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


ValueError: invalid literal for int() with base 10: '"'